In [43]:

import cv2 as cv
import numpy as np
import sys
import pickle as p

import math
import matplotlib.pyplot as plt
import nbinteract as nbi


In [44]:
class KalmanFilter:

    kf = cv.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    def Estimate(self, coordX, coordY):
        ''' This function estimates the position of the object'''
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        return predicted

#Performs required image processing to get ball coordinated in the video
class ProcessImage:

    def DetectObject(self, traj, imgsize = (240, 320, 3), plot = False):

        
        # Create Kalman Filter Object
        kfObj = KalmanFilter()
        predictedCoords = np.zeros((2, 1), np.float32)
        
        pos = []
        
        i = 0
        
        height, width = imgsize[0], imgsize[1]
        
        while(i < len(traj)):
            
            frame = np.zeros(imgsize)
                        
            [ballX, ballY] = traj[i]    
            
            predictedCoords = kfObj.Estimate(ballX, ballY)
            pos.append([predictedCoords[0], predictedCoords[1]])
            
            
            i += 1

        return pos

#Main Function
def main():
    processImg = ProcessImage()
    sine_imgsize = (480, 640, 3)
    sine = [[150 + 2*i, 240 + int(math.sin(i*(8*math.pi)/80) * 100) ] for i in range(80)]
        
    pos=processImg.DetectObject(sine, sine_imgsize, False)
    return pos,sine

def norme(t):
    
    
    pos,traj=main()

    '''Returns 1000 points drawn at random fron N(mean, sd)'''
    
    traj_x = [traj[i][0] for i in range(len(traj[0:t]))]
    traj_y = [traj[i][1] for i in range(len(traj[0:t]))]
    err= [((traj[i][1]-pos[i][1])**2 + (traj[i][0]-pos[i][0])**2)  for i in range(len(traj[0:t]))]
    pred_x = [pos[i][0] for i in range(len(pos[0:t]))]
    pred_y = [pos[i][1] for i in range(len(pos[0:t]))]

    plt.plot(traj_x[1:], traj_y[1:],'r',label='Actual')
    plt.plot(pred_x[:-1], pred_y[:-1],'y',label='Predicted')
    plt.show()
    return err




In [45]:

nbi.hist(norme,t=(2, 80))
